In [1]:
import xarray as xr 
import matplotlib.pylab as plt
import my_funs

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/noaa_nc/"


var_ts = xr.open_dataarray(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing.nc"
)

var_ts= var_ts.rename({"latitude":"lat","longitude":"lon"})
var_ts = var_ts[:,100:150,400:450]
# var_ts = var_ts[:,0:10,0:10]



In [55]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, TheilSenRegressor
import pymannkendall as mk


def _theilsen_mannkendall(y):
    x = np.arange(len(y)).reshape(-1, 1)
    if np.isnan(y).all():
        return np.nan, np.nan, np.nan

    I = np.where(np.isnan(y))
    if len(I[0]) > 1:
        return np.nan, np.nan, np.nan

    yy = np.delete(y, I)
    x = np.arange(len(yy)).reshape(-1, 1)
    result = mk.original_test(yy)
    # reg = TheilSenRegressor(random_state=0).fit(x, yy)
    # return result.slope, result.p, result.h
    # return result.slope
    return float(result.slope), float(result.p), float(result.h)

def est_trend(xrd, method, **kwargs):
    if method == "theilsen":
        cof, p, h = xr.apply_ufunc(
            _theilsen_mannkendall,
            xrd,
            input_core_dims=[["time"]],
            output_core_dims=[[], [], []],
            # dask="parallelized",
            # output_dtypes=float,
            vectorize=True,
        )
        ds_out = cof.to_dataset()
        ds_out["p_value"] = p
        ds_out["h"] = h
        return ds_out


In [2]:
ds = my_funs.est_trend(var_ts,"theilsen")